In [1]:
import tweepy
import numpy as np
import pandas as pd
import json

### Use Twitter API to fetch the tweets containing the hashtag #reopen

In [2]:
with open('api.json', 'r') as json_file:
    twitter_api = json.load(json_file)

consumer_key = twitter_api['consumer_key']
consumer_secret = twitter_api['consumer_secret']
access_token = twitter_api['access_token']
access_token_secret = twitter_api['access_token_secret']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit= True)

In [3]:
#Create class MyStreamListener inheriting from StreamListener and overriding on_status.
class StreamListener(tweepy.StreamListener):
    
    #Constructor in python
    def __init__(self, api=None):
        
        super(StreamListener, self).__init__()
        self.num_tweets = 0
        self.file = open("tweets.txt", "w")
        
    def on_status(self, status):

        if ('RT @' not in status.text):
            print(status.text)
            tweet = status._json
            self.file.write( json.dumps(tweet) + '\n' )
            self.num_tweets += 1
            print(self.num_tweets)
        if self.num_tweets < 50:
            return True
        else:
            return False
        self.file.close()

    #In order to close the stream you need to return False from on_data(), or on_status().
    #Because tweepy.Stream() runs a while loop itself, you don't need the while loop in on_data()
    def on_error(self, status_code):
        if status_code == 420:
            return False

stream_listener = StreamListener()
stream = tweepy.Stream(auth=api.auth, listener=stream_listener)
stream.filter(track=['#reopen', '#Reopen', '#ReOpen', '#Reopening', '#reopening'])

### While Filter API can fetch streaming data, Search API is a more efficient way for my project 

In [3]:
file = open("twitter.txt", "w")

# Search the matched tweets within the recent 7 days
cursor = tweepy.Cursor(api.search, q='#reopen OR #reopening', count= 100, result_type = 'recent', lang='en',
                        tweet_mode='extended').items()

for status in cursor:
    
    """status is json which is an object so hasattr() works but the method not work for dictionary """
    
    if not hasattr(status, "retweeted_status"):  # Check if Retweet
        
        tweet = status._json
        file.write(json.dumps(tweet) + '\n')

file.close()  